In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split 

In [7]:

# Caminho para o dataset
file_path = "../data/AmesHousing.csv"

# Carregar os dados
data = pd.read_csv(file_path)

# Escolhendo qual é o alvo
target = data.SalePrice
